In [4]:
import os
import glob
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import json
from keras.preprocessing import sequence, image
from keras.preprocessing.image import array_to_img, save_img, img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from keras.models import model_from_json
from keras.layers import (
    Flatten,
    Dense,
    Input,
    Activation,
    BatchNormalization,
    Conv2D,
    MaxPool2D,
    Dropout,
    UpSampling2D,
    Lambda,
)

from keras.layers import ReLU, Reshape, Conv2DTranspose, Concatenate, Multiply
from keras.models import Model

from keras.optimizers import Adam
from keras.losses import binary_crossentropy, categorical_crossentropy
from keras import backend as K

from keras.callbacks import ModelCheckpoint, EarlyStopping
from collections import Counter

from evaluation_metrics.multilabel.example_based import (
    hamming_loss,
    example_based_accuracy,
    example_based_precision,
    example_based_recall,
)

from evaluation_metrics.multilabel.label_based import (
    accuracy_macro,
    precision_macro,
    recall_macro,
    accuracy_micro,
    precision_micro,
    recall_micro,
)

from evaluation_metrics.multilabel.alpha_score import alpha_score
from data_loading.load_data import get_tile_data

%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [2]:
##loading train and testing data

data_directory = "../data/context_data/"
json_directory = "../data/json_files_trimmed_features/"
data = get_tile_data(data_directory, json_directory)
print("\nThe size of total data is", data.shape)
data = shuffle(data)

# split into train-test
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.10, random_state=42)

print("\nThe size of the train data is ", train_data.shape)
print("The size of the test data is ", test_data.shape)

Games detected in the parent folder ['lode_runner', 'kid_icarus', 'megaman', 'smb', 'loz']
Current Game lode_runner
Reading mappings
Json File Loaded
Reading Sprite Data From ../data/context_data/lode_runner
Current Game kid_icarus
Reading mappings
Json File Loaded
Reading Sprite Data From ../data/context_data/kid_icarus
Current Game megaman
Reading mappings
Json File Loaded
Reading Sprite Data From ../data/context_data/megaman
Current Game smb
Reading mappings
Json File Loaded
Reading Sprite Data From ../data/context_data/smb
Current Game loz
Reading mappings
Json File Loaded
Reading Sprite Data From ../data/context_data/loz

The size of total data is (25394, 5)

The size of the train data is  (22854, 5)
The size of the test data is  (2540, 5)


<IPython.core.display.Javascript object>

In [5]:
# load the multilabel binarizer
import pickle

with open("model_tokenizer.pickle", "rb") as handle:
    mlb = pickle.load(handle)
print("Feature Dictionary Loaded")
total_features = len(mlb.classes_)
print("The feature dictionary has size", total_features)
display("Features", mlb.classes_)

# load entire autoencoder architecture
json_file = open("autoencoder_model.json", "r")
loaded_model_json = json_file.read()
json_file.close()
ae_sep_output = model_from_json(loaded_model_json)
ae_sep_output.load_weights("autoencoder_model.h5")
print("Loaded Entire Autoencoder Model from the Disk")

# load the encoding architecture and weights
json_file = open("encoder_model.json", "r")
loaded_model_json = json_file.read()
json_file.close()
encoding_model = model_from_json(loaded_model_json)
encoding_model.load_weights("encoder_model.h5")
print("Loaded Encoder Model from the Disk")

# load the decoding architecture and weights
json_file = open("decoder_model.json", "r")
loaded_model_json = json_file.read()
json_file.close()
decoding_model = model_from_json(loaded_model_json)
# load weights into new model
decoding_model.load_weights("decoder_model.h5")
print("Loaded Decoder Model from the Disk")

Feature Dictionary Loaded
The feature dictionary has size 13


'Features'

array(['block', 'breakable', 'climbable', 'collectable', 'element',
       'empty', 'hazard', 'moving', 'openable', 'passable', 'pipe',
       'solid', 'wall'], dtype=object)

Loaded Entire Autoencoder Model from the Disk
Loaded Encoder Model from the Disk
Loaded Decoder Model from the Disk


<IPython.core.display.Javascript object>

In [7]:
# Build Input Output Training Batches
print("Building Training Batches")

"""Note : Add Generators"""
train_image_batch = []
for train_path in train_data["image_path"]:
    tile = image.load_img(train_path, target_size=(48, 48))
    tile_sprite = image.img_to_array(tile)
    train_image_batch.append(tile_sprite)
train_image_batch = np.array(train_image_batch)
train_text_batch = []
for i in range(len(train_data["features"])):
    text_ = mlb.transform(train_data["features"][i : i + 1])
    train_text_batch.append(text_)
train_text_batch = np.array(train_text_batch).reshape(
    train_data.shape[0], total_features
)

output_image_batch = []
for i in range(len(train_image_batch)):
    current_image = train_image_batch[i]
    current_image_centre = train_image_batch[i][16 : 16 + 16, 16 : 16 + 16, :]
    output_image_batch.append(current_image_centre)
output_image_batch = np.array(output_image_batch)
output_text_batch = []
for i in range(len(train_text_batch)):
    current_text = train_text_batch[i]
    output_text_batch.append(current_text)
output_text_batch = np.array(output_text_batch)
print("Training Data Ready")
print("Train Image batch shape", train_image_batch.shape)
print("Train Text batch shape", train_text_batch.shape)
print("Output Image batch shape", output_image_batch.shape)
print("Output Text batch shape", output_text_batch.shape)


# Build Input Output Test Batches
print("Building Testing Batches")
"""Note : Add Generators"""
test_image_batch = []
for test_path in test_data["image_path"]:
    tile = image.load_img(test_path, target_size=(48, 48))
    tile_sprite = image.img_to_array(tile)
    test_image_batch.append(tile_sprite)
test_image_batch = np.array(test_image_batch)
test_text_batch = []
for i in range(len(test_data["features"])):
    text_ = mlb.transform(test_data["features"][i : i + 1])
    test_text_batch.append(text_)
test_text_batch = np.array(test_text_batch).reshape(test_data.shape[0], total_features)
print("\n\nTesting Data Ready")
print("Train Image batch shape", test_image_batch.shape)
print("Train Text batch shape", test_text_batch.shape)

Building Training Batches
Training Data Ready
Train Image batch shape (22854, 48, 48, 3)
Train Text batch shape (22854, 13)
Output Image batch shape (22854, 16, 16, 3)
Output Text batch shape (22854, 13)
Building Testing Batches


Testing Data Ready
Train Image batch shape (2540, 48, 48, 3)
Train Text batch shape (2540, 13)


<IPython.core.display.Javascript object>

In [9]:
predicted_image, predicted_text = ae_sep_output.predict(
    [test_image_batch, test_text_batch]
)
y_pred = [np.where(text > 0.5, 1, 0) for text in predicted_text]
y_pred = np.array(y_pred)
print("Predicted Y is Ready. Shape : ", y_pred.shape)

y_true = test_text_batch
y_true = np.array(y_true)
print("True Y is Ready. Shape :", y_true.shape)

Predicted Y is Ready. Shape :  (2540, 13)
True Y is Ready. Shape : (2540, 13)


<IPython.core.display.Javascript object>

In [10]:
true_image = []
for i in range(len(test_image_batch)):
    current_image = test_image_batch[i]
    current_image_centre = test_image_batch[i][16 : 16 + 16, 16 : 16 + 16, :]
    true_image.append(current_image_centre)
true_image = np.array(true_image)
print("Predicted Array shape ", predicted_image.shape)
print("True Array shape ", true_image.shape)

mse_dist = []
for idx in range(len(true_image)):
    y_true_image = true_image[idx]
    y_true_image = y_true_image.reshape(16, 16, 3)

    y_pred_image = predicted_image[idx]
    y_pred_image = y_pred_image.reshape(16, 16, 3)

    mse_dist.append(np.mean(np.subtract(y_true_image, y_pred_image) ** 2))

print("Mean MSE", np.mean(mse_dist))
print("Median MSE", np.median(mse_dist))


def valid_divide(num, den):
    count = 0
    result = {}
    for idx in range(len(num)):
        if num[idx] == den[idx] == 0:
            continue
        elif num[idx] != 0 and den[idx] != 0:
            result[idx] = num[idx] / den[idx]
            count += 1
        elif num[idx] != 0 and den[idx] == 0 or num[idx] == 0 and den[idx] != 0:
            count += 1
            result[idx] = 0.0
    return result, count


print("\nMacro Label Based Precision", precision_macro(y_true, y_pred))
print("Macro Label Based Recall", recall_macro(y_true, y_pred))
print("Macro Label Based Accuracy", accuracy_macro(y_true, y_pred))

print("\nMicro Label Based Precision", precision_micro(y_true, y_pred))
print("Micro Label Based Recall", recall_micro(y_true, y_pred))
print("Micro Label Based Accuracy", accuracy_micro(y_true, y_pred))

print("\nExample Based Precision", example_based_precision(y_true, y_pred))
print("Example Based Recall", example_based_recall(y_true, y_pred))
print("Example Based Accuracy", example_based_accuracy(y_true, y_pred))

Predicted Array shape  (2540, 16, 16, 3)
True Array shape  (2540, 16, 16, 3)
Mean MSE 266.4628
Median MSE 36.328003

Macro Label Based Precision 0.9997858647027486
Macro Label Based Recall 0.9941111330011694
Macro Label Based Accuracy 0.9938972253123506

Micro Label Based Precision 0.9993521917512417
Micro Label Based Recall 0.998705222270177
Micro Label Based Accuracy 0.9980590899288333

Example Based Precision 0.9994094488188976
Example Based Recall 0.9990157480314961
Example Based Accuracy 0.9986876640419947


<IPython.core.display.Javascript object>